In [1]:
# bc018, bc016, bc015, bc011, bc010,

import iris
import os
import glob as glob
import datetime as datetime
import iris.coord_categorisation as cat
import sys
import iris.plot as iplt

# Set up path to root directory
root_fp = "/nfs/a319/gy17m2a/PhD/"
os.chdir(root_fp)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

# ### Load necessary spatial data
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})

# ### Establish the corresponding ensemble member numbers
em_matching_dict = {'01':'bc005', '04': 'bc006', '05': 'bc007', '06':'bc009',  '07':'bc010', 
                    '08': 'bc011', '09':'bc013', '10': 'bc015', '11': 'bc016', '12': 'bc017', '13':'bc018', '15':'bc012'}
em_matching_dict = {'01':'bb189', '04': 'bb192' ,'06':'bb198', '07':'bb201', '08':'bb204', '09':"bb211", '10':"bb216", '11':'bb219',
                   '12':'bb222','13':'bb225', '15':'bb208'}

resolution = '2.2km_original'
yrs_range = "2002_2020"

for em_1hr in ['01']:
    em_30mins = em_matching_dict[em_1hr]

    for yr in range(2061,2062):
        for month_num in ['01', '02', '03', '04', '05', '06','07','08','09', '10','11','12']:
            print(f"em{em_1hr}, yr {yr}, month {month_num}")
            if (os.path.isfile(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/{resolution}/{yrs_range}/{em_30mins}/{em_30mins}a.pr{yr}{month_num}.nc")):
                print("already exists")
            else:
                print(f"Running for month {month_num} in {yr}, for {em_1hr} (which equatees to {em_30mins})")

                ####################################################### 
                #######################################################
                ## Get one month of data - HOURLY
                ####################################################### 
                #######################################################
                ### Get a list of filenames for hourly data
                general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_original/{em_1hr}/{yrs_range}/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
                filenames_1hr = []
                for filename in glob.glob(general_filename_1hr):
                        filenames_1hr.append(filename)

                # ### Load in the data and remove the ensemble member dimension
                monthly_cubes_list_1hr = iris.load(filenames_1hr)
                cube_1hr = monthly_cubes_list_1hr[0]
                cube_1hr = cube_1hr[0,:,:,:]


em01, yr 2061, month 01
already exists
em01, yr 2061, month 02
already exists
em01, yr 2061, month 03
already exists
em01, yr 2061, month 04
already exists
em01, yr 2061, month 05
already exists
em01, yr 2061, month 06
already exists
em01, yr 2061, month 07
already exists
em01, yr 2061, month 08
already exists
em01, yr 2061, month 09
already exists
em01, yr 2061, month 10
already exists
em01, yr 2061, month 11
already exists
em01, yr 2061, month 12
already exists


In [26]:
em_1hr = '12'
em_matching_dict = {'01':'bc005', '04': 'bc006', '05': 'bc007', '06':'bc009',  '07':'bc010', 
                    '08': 'bc011', '09':'bc013', '10': 'bc015', '11': 'bc016', '12': 'bc017', '13':'bc018', '15':'bc012'}
em_30mins = em_matching_dict[em_1hr]
yrs_range = '2002_2020'
yr=2000

In [29]:
general_filename_30mins = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/{yrs_range}/{em_30mins}/{em_30mins}a.pr{yr}{month_num}*'
filenames_first30mins = []
for filename_30mins in glob.glob(general_filename_30mins):
    filenames_first30mins.append(filename_30mins)
filenames_first30mins

['/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000114.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000121.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000102.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000128.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000112.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000106.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000104.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000113.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000124.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000127.pp.zip',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020

In [30]:
# ### Load in the data 
monthly_cubes_list_30mins = iris.load( '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc017/bc017a.pr20000102.pp')

# Equalise
for cube in monthly_cubes_list_30mins:
    for attr in ['forecast_period', 'forecast_reference_time']:
        if attr in cube.attributes:
            del cube.attributes[attr]


monthly_cube_30mins = monthly_cubes_list_30mins.concatenate_cube()      
monthly_cube_30mins

<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (time: 24; grid_latitude: 654; grid_longitude: 532)>